# Building an End-to-end Project
## DEBUG WORKFLOW
### 1. Node package manager and dependancies
1. Initialise node package manager
    * `npm init -y`
2. Inspect the `dependencies` in `package.json` 
    * cor, dotenv, express, pg
    `npm install cors dotenv express pg`
3. `nodemon` requires developer dependancies
    * `npm install nodemon -d -y`
4. `package.json`
    * Include the below in `scripts`
    * "dev": "nodemon -L server/index.js"
    * "setup-db": "node ./server/db/setup.js"
    * Make sure that there are commons seperating each line


### 2. Files and folder structure: MAIN FOLDER THNE THE FOLLOWING
* .env
* .gitignore
* app.js
* index.js
* package.json
* .\controllers\\[APP NOUN NAME].js
* .\models\\[PROPER NOUN SINGULAR].js
* .\routes\\[APP NOUN NAME].js
* .\db\
    * connect.js
    * [APP NOUNT PLURAL].sql
    * setup.js
```bash
mkdir controllers models routes db
touch app.js
touch .end
touch index.js
touch [APP NOUN NAME].js
mv [APP NOUN NAME].js ./controllers/
touch [PROPER NOUN SINGULAR].js
mv [PROPER NOUN SINGULAR].js ./models/
touch [APP NOUN NAME].js
mv [APP NOUN NAME].js ./routers/
touch connect.js [APP NOUN NAME].sql setup.js
mv connect.js [APP NOUN NAME].sql setup.js ./db/
```


### 3. DATABASE
To instansiate the tables to the database, use
```bash
npm run setup-db
```



#### setup.js
```js
require('dotenv').config()
const fs = require('fs')

const db = require('./connect')

const sql = fs.readFileSync('./server/db/[APP NAME].sql').toString()

db.query(sql)
    .then(data => {
        db.end('setup complete')
    })
    .catch (error => console.log(error))
```


#### connect.js
```js
const {Pool} = require("pg")

// set up database connection
const db = new Pool ({
    connectionString: process.env.DB_URL
})

// Make the database available in the wider database:
module.exports = db
```

#### SQL file
Read the detail query in full. Check for the following:
* DROP used at the top.
* CREATE TABLE
    * COMMAs used in the correct positions,
    * SEMICOLON used at the end of lines,
    * Primary Key,
    * Every columm has a datatype 
* INSERT INTO
    * COMMAs used in the correct positions,
    * SEMICOLON used at the end of lines

### Base files
#### app.js
```js
const express       = require('express')
const cors          = require('cors')
const logger        = require('./logger')
const router        = require('./routers/[APP NAME]')
const app           = express()

app.use(express.json())
app.use(cors())
app.use(logger)
app.use('/[APP NAME]', router)

module.exports  = app
```

#### index.js
```js
require("dotenv").config()

const app   = require("./app")
const port  = process.env.PORT

app.listen(port, () => {
    console.log(`server listening on port ${port}`)
})
```

#### .env
```txt
PORT=
DB_URL=
DB_TEST_URL=
```

### MODELS
* database is connected and is defined
* All attributes in the class have been instantiated.
  * if there are any default settings, make sure they are present.
* CHECK ALL SQL QUERIES - they need to reflect what is asked for:
  * SELECT for GETS
  * INSERT for POST
  * UPDATE for PATCH
  * DELETE for DESTROY
* Make sure there is capitalisation of the Class.

```js
const db = require("../db/connect")

class Country {
  constructor({ country_id, name, capital, population, languages, fun_fact, map_image_url }) {
    this.country_id = country_id
    this.name = name
    this.capital = capital
    this.population = population
    this.languages = languages
    this.fun_fact = fun_fact
    this.map_image_url = map_image_url
  }

  static async findByCountryName(countryName) {
    return await db.query("SELECT * FROM country WHERE LOWER(name) = LOWER($1);", [countryName])
  }

  static async getAll() {
    const response = await db.query("SELECT name FROM country;")
    if (response.rows.length === 0) {
      throw new Error("No countries available")
    }
    return response.rows.map((c) => new Country(c))
  }

  static async getOneByCountryName(countryName) {
    const response = await Country.findByCountryName(countryName)
    if (response.rows.length != 1) {
      throw new Error("Unable to locate country")
    }
    return new Country(response.rows[0])
  }

  static async create(data) {
    const { name, capital, population, languages } = data
    const existingCountry = await Country.findByCountryName(name)
    if (existingCountry.rows.length === 0) {
      const response = await db.query("INSERT INTO country (name, capital, population, languages) VALUES ($1, $2, $3, $4) RETURNING *;", [name, capital, population, languages])
      return new Country(response.rows[0])
    } else {
      throw new Error("A country with this name already exists")
    }
  }

  async update(data) {
    const { capital, population, languages, name } = data
    if(!capital || !population || !languages || !name){
        throw new Error("Both captial and population are needed")
    }
    const response = await db.query("UPDATE country SET capital = $1, population = $2, languages = $3, name = $4 WHERE LOWER(name) = LOWER($5) RETURNING *;", [capital, population, languages, name, this.name])
    return new Country(response.rows[0])
  }

  async destroy() {
    const response = await db.query("DELETE FROM country WHERE LOWER(name) = LOWER($1) RETURNING *;", [this.name])
    return new Country(response.rows[0])
  }
}

module.exports = Country

### CONTROLLER
* CHECK CORRECT STATUS CODES:
    * 2xx
        * 200 - The request succeeded.
        * 201 - The request succeeded, and a **new resource was created** as a result. POST REQUEST
        * 204 - There is no content to send for this request, but the headers are useful. DESTROY REQUEST
    * 4xx
        * 400 - BAD REQUEST
        * 404 - NOT FOUND
    * 500 - INTERNAL SERVER ERROR
* module.exports - make sure this has been typed correctly.

```js
const Country = require("../models/Country")

async function index(req, res) {
    try {
        const countries = await Country.getAll()
        res.status(200).json(countries)
    } catch (err) {
        res.status(500).json({ error: err.message})
    }
}

async function show(req, res) {
    try {
        let name = req.params.name
        const country = await Country.getOneByCountryName(name)
        res.status(200).json(country)

    } catch (err) {
        res.status(404).json({ error: err.message })
    }
}


async function create(req, res) {
    try {
        const data = req.body
        const newCountry = await Country.create(data)
        res.status(201).json(newCountry)
    } catch (err) {
        res.status(400).json({ error: err.message })
    }
}

async function update(req, res) {
    try {
        const name = req.params.name
        const data = req.body
        const country = await Country.getOneByCountryName(name)
        if(!data.name) {
            data.name = country.name
        }
        if(!data.capital) {
            data.capital = country.capital
        }
        if(!data.population) {
            data.population = country.population
        }
        if(!data.languages) {
            data.languages = country.languages
        }
        const result = await country.update(data)
        res.status(200).json(result)
    } catch (err) {
        res.status(404).json({ error: err.message })
    }
}

async function destroy(req, res) {
    try {
        const name = req.params.name
        const country = await Country.getOneByCountryName(name)
        const result = await country.destroy()
        res.status(204).end()
    } catch (err) {
        res.status(404).json({ error: err.message })
    }
}


module.exports = {
    index,
    show,
    create,
    update,
    destroy
}
```

### ROUTER
* though the names of the methods are *arbitary*, follow convention with RESTFUL and the respect HTML methods

```js
const { Router } = require("express")

const countryController = require("../controllers/countries")

const countryRouter = Router()

countryRouter.get("/", countryController.index)
countryRouter.get("/:name", countryController.show)
countryRouter.post("/", countryController.create)
countryRouter.patch("/:name", countryController.update)
countryRouter.delete("/:name", countryController.destroy)

module.exports = countryRouter
```


## Work flow of an API call
RESTFUL Routing
1. CLIENT sends REQUEST
2. APP.js then gets ROUTED.js
3. ROUTED.js sends data to the CONTROLLER.js
4. CONTROLLER requests the data MODEL.js
5. MODEL.js searchs DATABASE
6. DATABASE sends back information back to MODEL.js
7. MODEL.js sends data to CONTROLLER.js
8. CONTROLLER.js sends RESPONSE back to the CLIENT

MODEL $\Rightarrow$ CONTROLLER $\Rightarrow$ ROUTER


## Set up
Create the folder `end-to-end`
```bash
mkdir end-to-end
cd end-to-end
```

Install Git
```bash
git init
```



Install Node Package Manager
```bash
npm init -y
```


Create a .gitignore file
```bash
touch .gitignore
```
And include the following:
```txt
node_modules
.env
```


Create a .env file
```bash
touch .env
```

Install the following for building an API
```bash
npm i -D nodemon
npm i express cors dotenv pg
```

We can install the package:
```bash
npm pkg set scripts.dev="nodemon -L server/index.js"
```

## Database
### Setting the database up
Build the database (ERD) first, as it builds 
```bash
mkdir server
cd server
mkdir db
cd db
touch countries.sql
```

### Creating the database
```sql
DROP TABLE IF EXISTS country;

CREATE TABLE country (
    country_id INT GENERATED ALWAYS AS IDENTITY,
    name VARCHAR(100) NOT NULL,
    capital VARCHAR(100) NOT NULL,
    population INT NOT NULL,
    languages VARCHAR(100) NOT NULL,
    fun_fact VARCHAR(255),
    map_image_url VARCHAR(255),
    PRIMARY KEY (country_id)
);

INSERT INTO country (name, capital, population, languages, fun_fact, map_image_url)
VALUES
  ('Brazil', 'Brasília', 212559417, 'Portuguese', 'Brazil is the fifth largest country in the world by both land area and population.', 'https://upload.wikimedia.org/wikipedia/commons/thumb/0/05/Flag_of_Brazil.svg/1200px-Flag_of_Brazil.svg.png'),
  ('Mexico', 'Mexico City', 127575529, 'Spanish', 'Mexico is home to the world''s largest pyramid, the Great Pyramid of Cholula.', 'https://upload.wikimedia.org/wikipedia/commons/thumb/f/fc/Flag_of_Mexico.svg/1200px-Flag_of_Mexico.svg.png'),
  ('United States', 'Washington, D.C.', 329064917, 'English', 'The United States has the largest economy in the world.', 'https://upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/1200px-Flag_of_the_United_States.svg.png'),
  ('India', 'New Delhi', 1353000000, 'Hindi, English', 'India is the world''s largest democracy.', 'https://upload.wikimedia.org/wikipedia/en/thumb/4/41/Flag_of_India.svg/1200px-Flag_of_India.svg.png'),
  ('China', 'Beijing', 1409517397, 'Mandarin', 'China has the world''s largest population.', 'https://upload.wikimedia.org/wikipedia/commons/thumb/f/fa/Flag_of_the_People%27s_Republic_of_China.svg/1200px-Flag_of_the_People%27s_Republic_of_China.svg.png'),
  ('Russia', 'Moscow', 145934462, 'Russian', 'Russia is the largest country in the world by land area.', 'https://upload.wikimedia.org/wikipedia/en/thumb/f/f3/Flag_of_Russia.svg/1200px-Flag_of_Russia.svg.png'),
  ('Japan', 'Tokyo', 126860301, 'Japanese', 'Japan is home to the world''s largest fish market, Tsukiji Market.', 'https://upload.wikimedia.org/wikipedia/en/thumb/9/9e/Flag_of_Japan.svg/1200px-Flag_of_Japan.svg.png'),
  ('South Africa', 'Pretoria', 57779622, 'Afrikaans, English, Zulu, Xhosa, and others', 'South Africa has 11 official languages.', 'https://upload.wikimedia.org/wikipedia/commons/thumb/a/af/Flag_of_South_Africa.svg/1200px-Flag_of_South_Africa.svg.png'),
  ('Australia', 'Canberra', 24982688, 'English', 'Australia is the only country that is also a continent.', 'https://upload.wikimedia.org/wikipedia/en/thumb/8/88/Flag_of_Australia.svg/1200px-Flag_of_Australia.svg.png');
```

.env
```text
DB_URL=[URL]
```

Terminal in the `server > db`
```bash
touch setup.js
touch connect.js
```



In setup.js
```js
require('dotenv').config()
const fs = require('fs')

const db = require('./connect')

const sql = fs.readFileSync('./server/db/countries.sql').toString()

db.query(sql)
    .then(data => {
        db.end('setup complete')
    })
    .catch (error => console.log(error))
```


In connect.js - provides the connection between API and database
```js
const {Pool} = require("pg")

const db = new Pool ({
    connectionString: process.env.DB_URL
})

// Make the database available in the wider database:
module.exports = db
```

In package.json:
```json
"scripts": {
    "setup-db": "node .server/db/setup.js"
}
```
The database should have been created.

Check the webiste and run:
```sql
SELECT * FROM country
```

```bash
touch app.js
```

Inside app.js - we create some middleware:
```js
const express   = require('express')
const cors      = require('cors')

const app       = express()

app.use(cors())

module.exports  = app
```

create a logger middleware
```bash
logger.js
```

we use next function to move onto the next part of code.<br>
Without next the code will stop at that point.
```js
const logger = (req, res, next) => {
    console.log(req.method, res,originalUrl)
}

module.exports = logger
```

We export the function to `app.js`:

```js
// ...
const logger = require('./logger')

// And
app.use(logger)
```

```bash
touch index.js
```

`index.js:`
```js
require("dotenv").config()
const app   = require("./app")

const port  = process.env.PORT

app.listen(port, () => {
    console.log(`server listening on port ${port}`)
})
```

set the port in .env
```txt
PORT=3000
```

CRUD | address | restful
 --- | -- | --
GET | '/' | index
GET | '/:id' | show
POST | '/' | create
PATCH | '/:id' | update
DELETE | '/id/ | destroy

In the `SERVER` directory, setup the model, router, controller folders and files:

```bash
mkdir controllers models routers
touch controllers/fruits.js models/Fruits.js routers/fruits.js


in `model > country.js` which runs on a class, not on a specific instance

MAKE SURE THE CLASS IS THE SAME AS THE FILENAME
```js
const db = require('../db/connect')

class Country {
    constructor({country_id, name, capital, population, languages, fun_fact, map_image_url}) {
        this.country_id     = country_id
        this.name           = name 
        this.capital        = capital
        this.population     = population 
        this.languages      = languages
        this.fun_fact       = fun_fact 
        this.map_image_url  = map_image_url 
    }
}

module.exports = Country
```

instance are PATCH or DELETE

## METHODS
### GET ALL
Our first get method to call on the database
```js
    static async getAll() {
        const response = await db.query("SELECT name FROM country;")
        if(reponse.rows.length === 0) {
            throw new Error('No countries available')
        }
        // each element in the array is mapped into an object
        return response.rows.map(c => new Country(c))
    }
```


Now we go to the `controller > countries.js` to tell us how to use our call on the database<br>
USE THE CORRECT HTTP STATUS CODE

```js
const Country = require('../models/Country')

async function index(req, res) {
    try {
        const countries = await Country.getAll()
        res.status(200).json(countries)
    } catch (err) {
        res.status(500).json({error: err.message})
    }
}

module.exports = {
    index
}
```

router redirects all requests to the necessary endpoints

in `routers > countries.js`:
```js
const express = require('express')
const controller = require("../controllers/countries")

const router = express.Router()

router.get('/', controller.index)

module.exports = router

We need to add the route into `app.js`:
```js
const countryRouter = require('./routers/countries')
// ...
app.use('/', countryRouter)


To convert JSON to javascript for the application
```js
app.use(express.json())
```

test this on `POSTMAN`:

GET     https://localhost:3000/countries

## SHOW
### MODEL
SHOW by starting at the model.js
```js
static async getOneByCountryName(countryName) {
    const response = await db.query("SELECT * FROM country WHERE LOWER(name) = LOWER($1);", [countryName])
    if(fresponse.rows.length != 1) {
        throw new Error(`Unable to locate ${countryName}`)
    }
    return new Country(response.rows[0])
}
```


### CONTROLLER
```js
async function show (res, req) {
    try {
        let name = req.params.name
        const country = await Country.getOneByCountryName(name)
        res.status(200).json(country)
    } catch (err) {
        res.status(400).json({error: err.message})
    }
}

// EXPORT
module.exports = {
    index,
    show
}

### ROUTE
```js
router.get('/name', controller.getOneByCountryName)

```

## CREATE
### MODEL

**THIS WILL BE TESTED**
```js
static async create(data) {
    const {name, capital, population, languages, fun_fact} = data

    // we have a function that checks the existance of a single country
    const alreadyExists = await db.query("SELECT name FROM country WHERE LOWER(name) = LOWER($1)", [name])
    
    // CHECK THE ORDER OF THIS INSERT INTO
    if (alreadyExists.rows.length === 0 ) {
        const response = await db.query("INSERT INTO country (name, capital, population, languages) VALUES ($1, $2, $3, $4) RETURNING *;", [name, capital, population, languages])
        // Access the only element in the rows array and store object to new variable
            const countryData = response.rows[0]
        
        // Pass object into the class constructor and generate new instance
            const newCountryInstance = new Country(countryData)
        
        // Return that new instance to the controller
            return newCountryInstance
    } else {
        throw new Error(`Country ${alreadyExists} already exists`)
    }
}
```

### CONTROLLERS
```js
async function create(req, res) {
    try {
        const data = req.body
        const input = await Country.create(data)
        res.status(201).json(input)
    } catch (err) {
        res.status(400).json({error: err.message})
    }
}

module.exports = {
    index,
    show,
    create
}
```

### ROUTERS
**THIS WILL BE TESTED**
```js
router.post('/', controller.create)
```


## DESTROY - instance method
### MODEL
```js
async destroy () {
    const response = await db.query("DELETE FROM country WHERE LOWER(name) = LOWER($1);", [this.name])
    return new Country(response.rows[0])
}
```



### CONTROLLER
```js
async function destroy (req, res) {
    try {
        const name = req.params.name
        const instanceOfCountry = await Country.getOneByCountryName(name)
        const result = await instanceOfCountry.destroy()
        res.status(204).end(message: 'Country successfully destroyed')
        // res.sendStatus(204)
    } else (err) {
        res.status(404).json({error: err.message})
    }
}

module.exports = {
    index,
    show,
    create,
    destroy
}

### ROUTER
```js
router.get('/', controller.index)
router.get('/:name', controller.show)
router.post('/', controller.create)
router.delete('/:name', controller.destroy)
```